In [1]:
import os
os.environ["LVMCORE_DIR"] = "/Users/droryn/prog/lvm/lvmcore"
os.environ["LVM_MASTER_DIR"] = "/Users/droryn/work/LVM/data/sas/sdsswork/lvm/sandbox/calib"
os.environ["SAS_BASE_DIR"] = "/Users/droryn/work/LVM/data/sas"
#os.environ["LVM_DATA_S"] = "/Users/droryn/work/LVM/data/sas/sdsswork/data/lvm/lco"
os.environ["LVM_SPECTRO_REDUX"] = "/Users/droryn/work/LVM/data/redux"
ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")

In [2]:
import numpy as np
from astropy.table import Table
from astropy.io import fits

from lvmdrp import path
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import run_quickdrp as qdrp

from lvmdrp.functions import imageMethod

# define environment before importing any DRP related modules

SLITMAP = Table(drp.fibermap.data)

def _parse_ccd_section(section):
    """Parse a CCD section in the format [1:NCOL, 1:NROW] to python tuples"""
    slice_x, slice_y = section.strip("[]").split(",")
    slice_x = list(map(lambda str: int(str), slice_x.split(":")))
    slice_y = list(map(lambda str: int(str), slice_y.split(":")))
    slice_x[0] -= 1
    slice_y[0] -= 1
    return slice_x, slice_y

def do_for_quadrants(image_path, func, *args, **kwargs):
    with fits.open(image_path) as hdul:
        image = hdul['PRIMARY'].data
        error = hdul['ERROR'].data
        mask = hdul['BADPIX'].data
        mask = ~mask*~np.isfinite(image)*error<=0
        ivar = np.where(mask, 1.0/(error**2), 0.0)
        header = hdul[0].header
        q1x, q1y = _parse_ccd_section(header['HIERARCH AMP1 TRIMSEC'])
        q2x, q2y = _parse_ccd_section(header['HIERARCH AMP2 TRIMSEC'])
        q3x, q3y = _parse_ccd_section(header['HIERARCH AMP3 TRIMSEC'])
        q4x, q4y = _parse_ccd_section(header['HIERARCH AMP4 TRIMSEC'])
        f1 = func(image[q1y[0]:q1y[1],q1x[0]:q1x[1]], ivar[q1y[0]:q1y[1],q1x[0]:q1x[1]], *args, **kwargs)
        f2 = func(image[q2y[0]:q2y[1],q2x[0]:q2x[1]], ivar[q2y[0]:q2y[1],q2x[0]:q2x[1]], *args, **kwargs)
        f3 = func(image[q3y[0]:q3y[1],q3x[0]:q3x[1]], ivar[q3y[0]:q3y[1],q3x[0]:q3x[1]], *args, **kwargs)
        f4 = func(image[q4y[0]:q4y[1],q4x[0]:q4x[1]], ivar[q4y[0]:q4y[1],q4x[0]:q4x[1]], *args, **kwargs)

        filtered = image.copy()*0
        filtered[q1y[0]:q1y[1],q1x[0]:q1x[1]] = f1
        filtered[q2y[0]:q2y[1],q2x[0]:q2x[1]] = f2
        filtered[q3y[0]:q3y[1],q3x[0]:q3x[1]] = f3
        filtered[q4y[0]:q4y[1],q4x[0]:q4x[1]] = f4

        return image, filtered


## Guille's flats with the spectrograph enclosure open

In [ ]:
MJD = 60171
print(ORIG_MASTER_DIR)

In [ ]:
drp.get_frames_metadata(mjd=MJD, overwrite=True)

In [ ]:
frames_table = md.get_metadata(tileid="*", mjd=MJD)
frames_table.query("imagetyp == 'dark'", inplace=True)
frames_table.sort_values("camera", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]
frames_table

In [ ]:
#os.listdir(ORIG_MASTER_DIR)
masters_mjd = qdrp.get_master_mjd(MJD)
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
print(masters_path)

In [6]:
# define target folder for detrended pixelflats
target_dir = "/Users/droryn/work/LVM/data/redux/pixflats"
os.makedirs(target_dir, exist_ok=True)

dflat_paths = []
for pixflat in frames_table.to_dict("records"):
    flat_path = path.full("lvm_raw", camspec=pixflat["camera"], **pixflat)
    # print(flat_path)
    
    # output path
    cam = pixflat["camera"]
    exp = pixflat["expnum"]
    dflat_path = os.path.join(target_dir, f"lvm-dpixflat-{cam}-{exp}.fits")
    dflat_paths.append(dflat_path)
    
    # calib paths
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{cam}.fits")
    
    #imageMethod.preproc_raw_frame(in_image=flat_path, out_image=dflat_path, replace_with_nan=False)
    #imageMethod.detrend_frame(in_image=dflat_path, out_image=dflat_path, in_bias=mbias_path, reject_cr=False, replace_with_nan=False)


In [7]:
# Stack frames 
frames_table["dflat_path"] = dflat_paths

cam_groups = frames_table.groupby("camera")
for cam in cam_groups.groups:
    dflat_paths_cam = cam_groups.get_group(cam)["dflat_path"]
    
    # define output combined pixelflat path
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    
    #imageMethod.create_master_frame(in_images=dflat_paths_cam, out_image=mflat_path)

## Iterative smoothing

In [ ]:
# Create pixel flats
# p preproc, d detrended, m stacked, c final filtered
#
from scipy import ndimage as ndi
import scipy.interpolate


def median_nan(image, ivar, size=31):
    image_tmp = np.where(ivar>0, image, np.NaN)
    return scipy.ndimage.generic_filter(image_tmp, np.nanmedian, size=size)

def filtering(image, ivar, debug=False):

    minflat = 0.001
    min_flat_for_fit_mask = 0.99
    max_flat_for_fit_mask = 1.02

    # initial model
    smooth = median_nan(image, ivar, size=31)
    if debug:
        fits.writeto('testmodel0.fits', smooth, overwrite=True)

    # initial flat by dividing by smoothed image, masking only where we have no data
    flat  =  (ivar>0)*(smooth>minflat)*image/(smooth*(smooth>minflat)+(smooth<=minflat))
    flat  += (smooth<=minflat)|(ivar<=0)  # set flat to 1 where masked
    if debug:
        fits.writeto('testflat0.fits', flat, overwrite=True)

    # dilate the mask, increasing sigma until not too large
    err = np.sqrt(1./(ivar+(ivar==0)))/(smooth*(image>0)+(image<=0))  # error image
    for nsig in [3.,3.5,4.,5.,10.,20.]:
        mask = (flat<(min_flat_for_fit_mask-nsig*err))|(flat>(max_flat_for_fit_mask+nsig*err))
        mask = ndi.binary_dilation(mask)
        frac = np.sum(mask>0)/float(np.sum(ivar>0))
        if frac<0.05 :
            break
    print("Used nsig = {}, frac = {:4.3f}".format(nsig,frac))

    # https://github.com/desihub/desispec/blob/main/bin/desi_compute_pixel_flatfield#L619

    # now start iterating smoothing and filtering the flat, ignoring newly mased pixels in the smoothing
    mask = mask | (ivar==0)
    smooth = median_nan(image, ~mask, size=31)
    flat  =  (ivar>0)*(smooth>minflat)*image/(smooth*(smooth>minflat)+(smooth<=minflat))   # divide by model
    flat  += (smooth<=minflat)|(ivar<=0)  # set flat to 1 where no data

    if debug:
        fits.writeto('testmask.fits', mask.astype(int), overwrite=True)
        fits.writeto('testivar.fits', ivar, overwrite=True)
        fits.writeto('testmodel.fits', smooth, overwrite=True)
        fits.writeto('testflat.fits', flat, overwrite=True)

    return flat#, model

def filter_image(image_path):
    return do_for_quadrants(image_path, filtering)
    #return do_for_quadrants(image_path, median_nan, size=51)

cam_groups = frames_table.groupby("camera")
for cam in cam_groups.groups:
    if cam!='b1':
        continue
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    pixflat_path = os.path.join(target_dir, f"lvm-cpixflat-{cam}.fits")
    filt_path = os.path.join(target_dir, f"filt-lvm-cpixflat-{cam}.fits")

    with fits.open(mflat_path) as hdul:
        print("Reading :", mflat_path)
        
        image, filtered = filter_image(mflat_path)
        flat = image/filtered
        outf = fits.HDUList(fits.PrimaryHDU(filtered))
        print("Writing :", filt_path)
        outf.writeto(filt_path, overwrite=True)
        outf.close()
        out = fits.HDUList(fits.PrimaryHDU(flat))
        print("Writing :", pixflat_path)
        out.writeto(pixflat_path, overwrite=True)
        out.close()


## Trivial Pixelflats

In [ ]:
# Create pixel flats
# p preproc, d detrended, m stacked, c final filtered
#
# TODO: deal with NaNs by pre-filtering -- scipy.median_filter spreads them around
# TODO: How to generate bad pixel mask?
#
from scipy import ndimage as ndi
from astropy.io import fits
from multiprocessing.pool import ThreadPool as Pool

def filter_image(image_path, size=30, mode='nearest'):
    with fits.open(mflat_path) as hdul:
        image = hdul[0].data
        header = hdul[0].header
        q1x, q1y = _parse_ccd_section(header['HIERARCH AMP1 TRIMSEC'])
        q2x, q2y = _parse_ccd_section(header['HIERARCH AMP2 TRIMSEC'])
        q3x, q3y = _parse_ccd_section(header['HIERARCH AMP3 TRIMSEC'])
        q4x, q4y = _parse_ccd_section(header['HIERARCH AMP4 TRIMSEC'])
        filtered = np.zeros(image.shape, dtype=np.float32)
        filtered[q1y[0]:q1y[1],q1x[0]:q1x[1]] = ndi.median_filter(image[q1y[0]:q1y[1],q1x[0]:q1x[1]], size=size, mode=mode)
        filtered[q2y[0]:q2y[1],q2x[0]:q2x[1]] = ndi.median_filter(image[q2y[0]:q2y[1],q2x[0]:q2x[1]], size=size, mode=mode) 
        filtered[q3y[0]:q3y[1],q3x[0]:q3x[1]] = ndi.median_filter(image[q3y[0]:q3y[1],q3x[0]:q3x[1]], size=size, mode=mode)
        filtered[q4y[0]:q4y[1],q4x[0]:q4x[1]] = ndi.median_filter(image[q4y[0]:q4y[1],q4x[0]:q4x[1]], size=size, mode=mode)
        #filtered = ndi.median_filter(image, size=30, mode='nearest')
        return image, filtered

def job(mflat_path, pixflat_path):
    print("Reading :", mflat_path)
    image, filtered = filter_image(mflat_path, size=50, mode='nearest')
    flat = image/filtered
    flat = np.where((flat>0.01)*(np.isfinite(flat)), flat, 1.0)
    # outf = fits.HDUList(fits.PrimaryHDU(filtered))
    # print("Writing :", filt_path)
    # outf.writeto(filt_path, overwrite=True)
    # outf.close()
    out = fits.HDUList(fits.PrimaryHDU(flat))
    print("Writing :", pixflat_path)
    out.writeto(pixflat_path, overwrite=True)
    out.close()

cam_groups = frames_table.groupby("camera")
with Pool(4) as p:
    for cam in cam_groups.groups:
        mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
        pixflat_path = os.path.join(target_dir, f"lvm-cpixflat-{cam}.fits")
        filt_path = os.path.join(target_dir, f"filt-lvm-cpixflat-{cam}.fits")

        r = p.apply_async(job, (mflat_path, pixflat_path))
    p.close()
    p.join()

## Check by flatfielding original frames

In [ ]:
import scipy.ndimage as ndi
# define target folder for detrended pixelflats
target_dir = "/Users/droryn/work/LVM/data/redux/pixflats"

dflat_paths = []
for pixflat in frames_table.to_dict("records"):
    cam = pixflat["camera"]
    exp = pixflat["expnum"]
    dflat_path = os.path.join(target_dir, f"lvm-dpixflat-{cam}-{exp}.fits")    
    mflat_path = os.path.join(target_dir, f"lvm-mpixflat-{cam}.fits")
    with fits.open(mflat_path) as hdf:
        f = hdf[0].data

    with fits.open(dflat_path) as hdu:
        d = hdu['PRIMARY'].data
    
    d /= f
    m = ndi.median_filter(d, size=31)
    print(dflat_path, f'test-{cam}.fits')
    fits.writeto(f'test-{cam}.fits', d/m, overwrite=True)


## Nick's pixelflats taken in the lab

In [ ]:
drp.get_frames_metadata(mjd=59720, overwrite=True)
drp.get_frames_metadata(mjd=59724, overwrite=True)

In [ ]:
# define target folder for preprocessed pixelflats
# leave just preprocessed frames for one low, one high count exposure
# https://docs.google.com/spreadsheets/d/103BNxjlZ59Sob3jDO4EN1z6zp2q5YrYA6nTjGlZM6XY/edit#gid=349553156
target_dir = "/Users/droryn/work/LVM/data/redux/nickflats"
os.makedirs(target_dir, exist_ok=True)
MJD=59720

#masters_mjd = qdrp.get_master_mjd(MJD)
masters_mjd = 60142
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
print(masters_path)

frames_table = md.get_metadata(tileid="*", mjd=MJD)
frames_table.query("expnum>=3814 & expnum<=3843", inplace=True)
frames_table.sort_values("camera", inplace=True)
#frames_table

pflat_paths = []
dflat_paths = []
for pixflat in frames_table.to_dict("records"):
    flat_path = path.full("lvm_raw", camspec=pixflat["camera"], **pixflat)
    # print(flat_path)
    
    # output path
    cam = pixflat["camera"]
    exp = pixflat["expnum"]
    pflat_path = os.path.join(target_dir, f"lvm-ppixflat-{cam}-{exp}.fits")
    pflat_paths.append(pflat_path)
    dflat_path = os.path.join(target_dir, f"lvm-dpixflat-{cam}-{exp}.fits")
    dflat_paths.append(dflat_path)
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{cam}.fits")

    imageMethod.preproc_raw_frame(in_image=flat_path, out_image=pflat_path, assume_imagetyp="pixelflat", replace_with_nan=False)
    imageMethod.detrend_frame(in_image=pflat_path, out_image=dflat_path, in_bias=mbias_path, reject_cr=False, replace_with_nan=False)


In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.ndimage import binary_closing

d = '/Users/droryn/work/LVM/data/sas/sdsswork/lvm/sandbox/calib/pixelmasks/'
cam = ['b', 'r', 'z']
spec = ['1', '2', '3']
for c in cam:
    for s in spec:
        with fits.open(d+f'lvm-mpixflat-{c}{s}.fits') as f:
            d1 = f['PRIMARY'].data.astype(np.float32)
        mask = ((d1>1.1) | (d1<0.5)).astype(np.int8)
        # need a margin for this to work with edges
        margin=50
        larger_mask = np.ones((mask.shape[0]+2*margin,mask.shape[1]+2*margin),dtype=np.int8)
        for loop in range(1):
            larger_mask[margin:-margin,margin:-margin] = mask
            closed_mask = binary_closing(larger_mask, iterations=10, structure=np.ones([2, 2]).astype(np.int8))
            assert(closed_mask.shape==larger_mask.shape)
            mask|=closed_mask[margin:-margin,margin:-margin]
        fits.writeto(d+f'mask-{c}{s}.fits', mask, overwrite=True)